In [49]:
import sys
import os
import json

# Add the project root to sys.path to import src
# Adjust this path depending on where you run the notebook
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '23127011'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Added to path: {project_root}")

from src.parser import find_root_tex_file, LatexFlattener, LatexStructureBuilder, LatexContentProcessor
from src.processing import ContentDeduplicator

Added to path: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011


In [50]:
!pip install bibtexparser texsoup

In [51]:
from src.parser import LatexFlattener, LatexStructureBuilder, LatexContentProcessor, find_root_tex_file
from src.cleaner import ReferenceProcessor
from src.deduplicator import ReferenceDeduplicator, ContentDeduplicator, replace_citations_in_text


In [52]:
# Configuration
DATA_RAW_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data_raw_real'))

print(f"Data raw path: {DATA_RAW_PATH}")

if not os.path.exists(DATA_RAW_PATH):
    print("❌ Warning: Data raw path does not exist. Please check the path.")
else:
    print("✅ Data raw path found.")

Data raw path: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real
✅ Data raw path found.


In [ ]:
paper_base_id = '2403-00599'
versions_to_process = '2403-00599v1'


In [54]:

# 1. Cấu hình đường dẫn cho Version cụ thể
ver_path = os.path.join(DATA_RAW_PATH, paper_base_id, 'tex', versions_to_process)
print(f"📂 Testing directory: {ver_path}")

# 2. Bước tiên quyết: Flatten nội dung trước (vì ReferenceProcessor cần text phẳng)
root_file = find_root_tex_file(ver_path)


📂 Testing directory: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real\2403-00559\tex\2403-00559v2


In [55]:
tex_files = []
for root, dirs, files in os.walk(ver_path):
    # dirs[:] = [d for d in dirs if d not in BLOCKLIST_DIRS]
    print(root, dirs, files)
    for f in files:
        # all_files.add(f)
        if f.lower().endswith('.tex'):
            tex_files.append(os.path.join(root, f))

print("Found .tex files:", tex_files)
print(ver_path)

d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real\2403-00559\tex\2403-00559v2 [] ['PMNS-expansion.tex']
Found .tex files: ['d:\\Coding\\School\\Y3-K1\\Intro2DS\\DS - LAB 2\\Milestone2_Project\\data_raw_real\\2403-00559\\tex\\2403-00559v2\\PMNS-expansion.tex']
d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real\2403-00559\tex\2403-00559v2


In [56]:
print(root_file)

d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real\2403-00559\tex\2403-00559v2\PMNS-expansion.tex


In [57]:
# %%
# --- TEST RUNNER CHO REFERENCE PROCESSOR ---

if not root_file:
    print("❌ Error: Could not find root tex file.")
else:
    print(f"✅ Root file found: {os.path.basename(root_file)}")
    
    # Flatten
    flattener = LatexFlattener(root_file, paper_id=paper_base_id, version=versions_to_process, remove_references=False)
    flat_result = flattener.flatten()
    flat_content = flat_result['content']
    print(f"📄 Flattened content size: {len(flat_content)} characters")
    # print('\\bibliography{apssamp}' in flat_content)
# \bibliography{apssamp}% Produces the bibliography via BibTeX.
# 
    # 3. Khởi tạo và chạy ReferenceProcessor
    print("\n--- START REFERENCE PROCESSING ---")
    print(ver_path)
    ref_processor = ReferenceProcessor(paper_base_id, versions_to_process, ver_path)
    
    # Hàm này sẽ in ra log chi tiết về việc tìm thấy .bbl hay .bib và số lượng ref lọc được
    content_after_process, final_refs = ref_processor.process_references(flat_content)
    
    print("--- END REFERENCE PROCESSING ---\n")

    # 4. Kiểm tra kết quả đầu ra
    print(f"📊 Total Valid References: {len(final_refs)}")
    
    if len(final_refs) > 0:
        print("\n🔎 Sample Reference 1:")
        print(json.dumps(final_refs[0], indent=2, ensure_ascii=False))
        
        # Kiểm tra xem key của ref có đúng format không
        print(f"\nKey: {final_refs[0]['key']}")
        print(f"Source Type: {final_refs[0]['type']}")
    else:
        print("⚠️ No references found. Check if the paper actually has citations or if regex needs adjustment.")

✅ Root file found: PMNS-expansion.tex
📄 Flattened content size: 46990 characters

--- START REFERENCE PROCESSING ---
d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\data_raw_real\2403-00559\tex\2403-00559v2
   🔍 Scanning references for 2403-00559v2...
      Found 45 cited keys (Regex engine).
        -> Parsed 45 items from embedded_block
      Matched 45 references.
--- END REFERENCE PROCESSING ---

📊 Total Valid References: 45

🔎 Sample Reference 1:
{
  "key": "Cabibbo:1963yz",
  "raw_text": "N.~Cabibbo, Unitary Symmetry and Leptonic Decays, Phys.\\ Rev.\\ Lett.\\ {\\bf 10} (1963) 531.",
  "type": "bibitem",
  "source": "embedded_block"
}

Key: Cabibbo:1963yz
Source Type: bibitem


In [58]:
with open(f'references-{paper_base_id}-v2.json', 'w', encoding = 'utf-8') as f:
    json.dump(final_refs, f, indent=2, ensure_ascii=False)

In [59]:

print(flat_content)

\documentclass[12pt, a4paper]{article}
\usepackage{geometry, amsmath,amssymb,amsthm,amsxtra,overpic,bbm,bm,epsfig,sub
figure}

\usepackage{mathrsfs}
\usepackage{graphicx}
\usepackage{color}
\usepackage{comment}
\usepackage{epstopdf}
\usepackage{float}

\geometry{left=1.85cm,right=1.85cm,top=1.7cm,bottom=2.5cm}



\usepackage{slashed,stmaryrd}

\usepackage{multirow}


\renewcommand{\baselinestretch}{1.3}
\def\thefootnote{\fnsymbol{footnote}}



\addtolength{\arraycolsep}{-3pt} 

\begin{document}

\vspace{0.2cm}

\begin{center}
{\large\bf A new Wolfenstein-like expansion of lepton flavor mixing \\
towards understanding its fine structure}
\end{center}

\vspace{0.2cm}

\begin{center}
{\bf Zhi-zhong Xing$^{1,2}$}
\footnote{E-mail: xingzz@ihep.ac.cn}
\\
{\small $^{1}$Institute of High Energy Physics and School of Physical Sciences, \\
University of Chinese Academy of Sciences, Beijing 100049, China \\
$^{2}$Center of High Energy Physics, Peking University, Beijing 100871, China}
\end{center

In [60]:
# 1. Setup
paper_base_id = '2403-00530'
versions = ['2403-00530v0', '2403-00530v2'] # List các version
ref_deduplicator = ReferenceDeduplicator()
content_deduplicator = ContentDeduplicator()

# Lưu trữ nội dung phẳng tạm thời để parse sau khi dedup ref
temp_flat_contents = {} 

print("🚀 STEP 1: GATHERING & REFERENCE EXTRACTION")
for ver in versions:
    ver_path = os.path.join(DATA_RAW_PATH, paper_base_id, 'tex', ver)
    
    # A. Flatten
    root_file = find_root_tex_file(ver_path)
    if not root_file: continue
    
    flattener = LatexFlattener(root_file, paper_id=paper_base_id, version=ver,remove_references=False)
    flat_res = flattener.flatten()
    flat_text = flat_res['content']
    
    # B. Reference Extract (Dùng class mới bạn đã test)
    ref_proc = ReferenceProcessor(paper_base_id, ver, ver_path)
    flat_text, refs = ref_proc.process_references(flat_text)
    
    # C. Add to Dedup
    ref_deduplicator.add_references(ver, refs)
    
    # Lưu lại text gốc để tí nữa sửa
    temp_flat_contents[ver] = flat_text

print("\n🚀 STEP 2: TEXT REPLACEMENT & PARSING")
for ver in versions:
    if ver not in temp_flat_contents: continue
    
    # A. Lấy map thay thế (Old -> New)
    replacements = ref_deduplicator.get_replacements(ver)
    raw_text = temp_flat_contents[ver]
    
    # B. Sửa Key trong Text
    # Hàm này sẽ đổi \cite{hinton06} thành \cite{ref_0}
    updated_text = replace_citations_in_text(raw_text, replacements)
    
    print(f"   📝 Parsed {ver} with {len(replacements)} citation updates.")
    
    # C. Parse Structure (Dùng text ĐÃ SỬA)
    builder = LatexStructureBuilder(updated_text, paper_base_id, ver)
    root_node = builder.build_coarse_tree()
    
    # D. Clean Content
    processor = LatexContentProcessor(paper_base_id, ver)
    processor.process_tree(root_node)
    
    # E. Content Dedup
    content_deduplicator.process_version(ver, root_node)

print("\n🚀 STEP 3: EXPORT")

# 1. Export refs.bib
bib_str = ref_deduplicator.export_bib_string()
with open('refs.bib', 'w', encoding='utf-8') as f:
    f.write(bib_str)
print("✅ Exported refs.bib")

# 2. Export hierarchy.json
final_hier = content_deduplicator.get_final_json()
with open('hierarchy.json', 'w', encoding='utf-8') as f:
    json.dump(final_hier, f, indent=2, ensure_ascii=False)
print("✅ Exported hierarchy.json")

🚀 STEP 1: GATHERING & REFERENCE EXTRACTION
   🔍 Scanning references for 2403-00530v2...
      Found 92 cited keys (Regex engine).
        -> Parsed 92 items from main_rerev2025.bbl
      Matched 92 references.

🚀 STEP 2: TEXT REPLACEMENT & PARSING
   📝 Parsed 2403-00530v2 with 92 citation updates.

🚀 STEP 3: EXPORT
✅ Exported refs.bib
✅ Exported hierarchy.json
